In [1]:
import os
import pandas as pd

### (Easy) Q1. Military Branch Consumer Fraud

In [58]:
military_consumer_fraud_df = pd.read_csv("legal/raw/CSVs/2024_CSN_Fraud, Identity Theft, and Other Reports by Military Consumers.csv")

# extract branch fraud table
branch_fraud_df = military_consumer_fraud_df.iloc[7:14]
column_names = list(branch_fraud_df.iloc[0].to_dict().values())
branch_fraud_df.columns = column_names
branch_fraud_df = branch_fraud_df.iloc[1:]

# find branch w/max median fraud loss
max_branch, max_median_fraud_loss = None, None
for idx, branch_row in branch_fraud_df.iterrows():
    median_fraud_loss = int(branch_row['Median Fraud Loss'].replace("$","").replace(",",""))
    if max_branch is None or median_fraud_loss > max_median_fraud_loss:
        max_branch = branch_row['Military Branch']
        max_median_fraud_loss = median_fraud_loss
print(max_branch)

U.S. Space Force


### (Easy) Q2. States with Top Report Category == Identity Theft

In [119]:
df = pd.read_csv("legal/raw/CSVs/2024_CSN_State_Top_Ten_Report_Categories.csv")

# extract data
df = df.iloc[1:-4]
column_names = list(df.iloc[0].to_dict().values())
df.columns = [col.strip() for col in column_names]
df = df.iloc[1:]

# group by state, sort by # reports, and add to list of states w/Identity Theft as top category
identity_theft_states = []
for state, state_df in df.groupby('State'):
    state_df['# of Reports'] = state_df['# of Reports'].apply(lambda entry: int(entry.replace(",","")))
    state_df.sort_values(['# of Reports'], ascending=False, inplace=True)
    if state_df.iloc[0]['Category'] == 'Identity Theft':
        identity_theft_states.append(state)

print(sorted(identity_theft_states))

['Arizona', 'California', 'Massachusetts', 'Ohio', 'Oklahoma', 'Rhode Island']


### (Easy) Q3. States with "Prizes, Sweepstakes and Lotteries" in their top-10 report categories?

In [122]:
df = pd.read_csv("legal/raw/CSVs/2024_CSN_State_Top_Ten_Report_Categories.csv")

# extract data
df = df.iloc[1:-4]
column_names = list(df.iloc[0].to_dict().values())
df.columns = [col.strip() for col in column_names]
df = df.iloc[1:]

# group by state, check if "Prizes, Sweepstakes and Lotteries" is in categories
prizes_states = []
for state, state_df in df.groupby('State'):
    if "Prizes, Sweepstakes and Lotteries" in state_df['Category'].tolist():
        prizes_states.append(state)

print(len(prizes_states))

27


### (Hard) Q1. Gen. Pop. vs. Military

In [93]:
military_consumer_reports_df = pd.read_csv("legal/raw/CSVs/2024_CSN_Reports_by_Military_Consumers.csv")
report_categories_df = pd.read_csv("legal/raw/CSVs/2024_CSN_Report_Categories.csv", encoding="ISO-8859-1")

# extract military table
top_military_categories_df = military_consumer_reports_df.iloc[12:23]
column_names = list(top_military_categories_df.iloc[0].to_dict().values())
top_military_categories_df.columns = [col.strip() for col in column_names]
top_military_categories_df = top_military_categories_df.iloc[1:]

# extract report table
column_names = list(report_categories_df.iloc[1].to_dict().values())
report_categories_df.columns = [col.strip() for col in column_names]
report_categories_df = report_categories_df.iloc[2:-4]

# get pct of imposter scams in military
top_military_categories_df['# of Reports'] = top_military_categories_df['# of Reports'].apply(lambda elt: int(elt.replace(",","")))
total_military_reports = top_military_categories_df['# of Reports'].sum()
military_imposter_scams = top_military_categories_df.loc[top_military_categories_df.Category == 'Imposter Scams']['# of Reports'].iloc[0]
pct_military_imposter_scams = military_imposter_scams/total_military_reports

# get pct of imposter scams in general population
pct_general_imposter_scams = report_categories_df.loc[report_categories_df.Category == "Imposter Scams"]['Percentage'].iloc[0]
pct_general_imposter_scams = float(pct_general_imposter_scams.replace("%",""))/100.0

print(pct_military_imposter_scams > pct_general_imposter_scams)

True


### (Hard) Q2. Metro w/Largest Pct. Fraud within State 

In [141]:
import os

# get all filepaths
dir_path = 'legal/raw/CSVs/State MSA Fraud and Other data'
filenames = os.listdir(dir_path)
filepaths = [os.path.join(dir_path, name) for name in filenames]

# compute the metro area with the max pct. of fraud in its own state
max_fraud_metro_area, max_fraud_report_pct = None, 0.0
for idx, filepath in enumerate(filepaths):
    # extract the dataframe
    df = pd.read_csv(filepath)
    df = df.iloc[1:-4]
    column_names = list(df.iloc[0].to_dict().values())
    df.columns = [col.strip() for col in column_names]
    df = df.iloc[1:]

    # skip states with fewer than 5 metro areas
    if len(df) < 5:
        continue

    # compute pct. of total reports for each metro
    df['# of Reports'] = df['# of Reports'].apply(lambda entry: int(entry.replace(",","")))
    df['fraud_report_pct'] = df['# of Reports'] / df['# of Reports'].sum()
    df.sort_values(['fraud_report_pct'], ascending=False, inplace=True)

    # get largest fraud metro area for this state and its percent of reports
    state_max_fraud_metro_area = df.iloc[0]['Metropolitan Area']
    state_max_fraud_report_pct = df.iloc[0]['fraud_report_pct']
    if max_fraud_metro_area is None or state_max_fraud_report_pct > max_fraud_report_pct:
        max_fraud_metro_area = state_max_fraud_metro_area
        max_fraud_report_pct = state_max_fraud_report_pct

print(max_fraud_metro_area)

Washington-Arlington-Alexandria, DC-VA-MD-WV Metropolitan Statistical Area


### (Hard) Q3. Metro w/Largest Pct. Fraud within State

In [113]:
import os

# get all filepaths
fraud_dir_path = 'legal/raw/CSVs/State MSA Fraud and Other data'
fraud_filenames = os.listdir(fraud_dir_path)
fraud_filepaths = [os.path.join(fraud_dir_path, name) for name in fraud_filenames]

theft_dir_path = 'legal/raw/CSVs/State MSA Identity Theft Data'
theft_filenames = os.listdir(theft_dir_path)
theft_filepaths = [os.path.join(theft_dir_path, name) for name in theft_filenames]

# compute the number of fraud and identity theft reports in each metro area and return True
# if a metro area has more identity theft reports than fraud reports

# compute mapping from metro area to num fraud reports
metro_area_to_fraud_reports = {}
for idx, filepath in enumerate(fraud_filepaths):
    # extract the dataframe
    df = pd.read_csv(filepath)
    df = df.iloc[1:-4]
    column_names = list(df.iloc[0].to_dict().values())
    df.columns = [col.strip() for col in column_names]
    df = df.iloc[1:]

    # compute pct. of total reports for each metro
    df['# of Reports'] = df['# of Reports'].apply(lambda entry: int(entry.replace(",","")))
    metro_area_to_fraud_reports = {
        **pd.Series(df['# of Reports'], index=df['Metropolitan Area'].values).to_dict(),
        **metro_area_to_fraud_reports,
    }

# compute mapping from metro area to num identity theft reports
metro_area_to_identity_reports = {}
for idx, filepath in enumerate(theft_filepaths):
    # extract the dataframe
    df = pd.read_csv(filepath)
    df = df.iloc[1:-4]
    column_names = list(df.iloc[0].to_dict().values())
    df.columns = [col.strip() for col in column_names]
    df = df.iloc[1:]

    # compute pct. of total reports for each metro
    df['# of Reports'] = df['# of Reports'].apply(lambda entry: int(entry.replace(",","")))
    metro_area_to_identity_reports = {
        **pd.Series(df['# of Reports'], index=df['Metropolitan Area'].values).to_dict(),
        **metro_area_to_identity_reports,
    }

for metro_area, num_identity_reports in metro_area_to_identity_reports.items():
    num_fraud_reports = metro_area_to_fraud_reports.get(metro_area, None)
    if num_fraud_reports is not None and num_identity_reports > num_fraud_reports:
        print(metro_area)
        print(True)
        break